In [1]:
# {'pkts_dropped': 0.0,
#  'pkts_transmitted': 75.0,
#  'timestamp': '1626810037874447104',
#  'obj': 'simulation_pedestrian1',
#  'pos_x': '11.417197',
#  'pos_y': '37.027515',
#  'pos_z': '7.4369965',
#  'orien_x': '-0.0',
#  'orien_y': '0.0',
#  'orien_z': '0.9999752',
#  'orien_w': '0.0070461035',
#  'linear_acc_x': '',
#  'linear_acc_y': '',
#  'linear_acc_z': '',
#  'linear_vel_x': '',
#  'linear_vel_y': '',
#  'linear_vel_z': '',
#  'angular_acc_x': '',
#  'angular_acc_y': '',
#  'angular_acc_z': '',
#  'angular_vel_x': '',
#  'angular_vel_y': '',
#  'angular_vel_z': '',
#  'pkts_buffered': 0.0,
#  'bit_rate': 4949598.859792932,
#  'chosen_ue': 'simulation_pedestrian1',
#  'packets': 14627.0,
#  'channel_mag': array(0.00890296)}

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import tqdm

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import copy

import gym

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

In [3]:
device = torch.device("cpu")

# Gym Environment

In [4]:
import caviar_tools
from beamselect_env import BeamSelectionEnv

### Hyper Params

In [5]:
reward_type = 'test'    # 'test' or 'train'
epi = [0,10] #[start,end] 
epi_val = [500,549]


gym_env = BeamSelectionEnv(epi, reward_type)
gym_env_val = BeamSelectionEnv(epi_val)

n_steps = caviar_tools.linecount(epi)
n_steps_val = caviar_tools.linecount(epi_val)

/home/mohit/anaconda3/envs/ITU/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
n_steps

68700

Observation Space : X,Y,Z,pkts_dropped,pkts_transmitted,pkts_buffered,bit_rate

Action Space : [3,64] -> [UE,Possible beams]

## Replay Memory

In [7]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## DQN

In [8]:
class DQN(nn.Module):

    def __init__(self, inputs:int=7, outputs:int=64*3):
        super().__init__()

        self.linear = nn.Sequential(
            self.create_linear(inputs,16),
            self.create_linear(16, 32),
            self.create_linear(32,64),
            self.create_linear(64,256)
        )

        self.value_linear = self.create_linear(256,1)
        self.action_linear = self.create_linear(256,outputs)

    
    def create_linear(self,inp:int,out:int)-> nn.Module:
        return nn.Sequential(
            nn.Linear(inp,out),
            nn.ELU()
            # nn.BatchNorm1d(out)
        )
    
    def forward(self, x):
        x = x.to(device)
        x = self.linear(x)

        value = self.value_linear(x)
        adv = self.action_linear(x)


        advAverage = torch.mean(adv, dim=1, keepdim=True)
        
        out = value + adv - advAverage
        return out

        

In [9]:
tmp = DQN()
sample = torch.rand((1,7))
tmp(sample).shape

torch.Size([1, 192])

## Hyperparams

In [10]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.3

# It depends on overall number of steps, basic intitution is that
# once steps_done == EPS_DECAY then the probablity of choosing 
# random action is 33%; considering EPS_END is zero
# As for ep = [0,10]; approx ep is 80k therefore exploration can be reduced to 33% around 50k
# Also because of this factor smoothed accuracy matters more for training then seeing the average
EPS_DECAY = n_steps*0.3

TARGET_UPDATE = 1000
VAL_STEP = 30000

EPOCH = 1

## Action

In [11]:
policy_net = DQN(11, 192).to(device)
target_net = DQN(11, 192).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

n_actions = 64*3
steps_done = 0

def select_action(state,val:bool=False):
    if val:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            flattened_action = policy_net(state).max(dim = 1).indices
            return torch.tensor([[flattened_action]], device=device, dtype=torch.long)
    else:
        global steps_done
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * \
            math.exp(-1. * steps_done / EPS_DECAY)
        steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                # t.max(1) will return largest column value of each row.
                # second column on max result is index of where max element was
                # found, so we pick action with the larger expected reward.
                flattened_action = policy_net(state).max(dim = 1).indices
                return torch.tensor([[flattened_action]], device=device, dtype=torch.long)
        else:
            flattened_action = random.randrange(n_actions)
            return torch.tensor([[flattened_action]], device=device, dtype=torch.long)


## Optimize Model

In [12]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

## Training loop

In [13]:
# Tensorboard
log_dir = './mini_logs_with_test/target_10' 

writer = SummaryWriter(log_dir=log_dir)

In [14]:
def val(train_step:int,gym_env_val:gym.Env,n_steps_val:int=n_steps_val,
        writer:SummaryWriter=writer):

    state = torch.zeros((1,11), dtype=torch.float32)
    ovr_reward = 0
    for i_episode in tqdm.tqdm_notebook(range(n_steps_val),desc='Validation'):
        # Select and perform an action
        action = select_action(state,val=True)
        # Observe new state
        next_state, reward, done, info = gym_env_val.step([action.item()//64, action.item()%64])


        ovr_reward+=reward.item()

        next_state = next_state.astype(np.float32).reshape(1, next_state.shape[0])
        next_state = torch.tensor(next_state)
        reward = torch.tensor([reward], device=device).float()

        # Move to the next state
        state = next_state
        
    writer.add_scalar('Val Overall Reward',ovr_reward,train_step)
    writer.add_scalar('Val Average Rewad',(ovr_reward/n_steps_val),train_step)

    print(f'Validation Overall reward = {ovr_reward:.2f}. ' \
        f' Validation Average Reward = {ovr_reward/n_steps_val:.4f}')
    gym_env_val.close()
    

In [15]:
# Initialize the environment and state
for i in range(0,EPOCH):
    gym_env.reset()
    state = torch.zeros((1,11), dtype=torch.float32)
    ovr_reward = 0
    for i_episode in tqdm.tqdm_notebook(range(n_steps),desc='Train'):
        # Select and perform an action
        action = select_action(state)
        # Observe new state
        next_state, reward, done, info = gym_env.step([action.item()//64, action.item()%64])


        ovr_reward+=reward.item()

        next_state = next_state.astype(np.float32).reshape(1, next_state.shape[0])
        next_state = torch.tensor(next_state)
        reward = torch.tensor([reward], device=device).float()

        writer.add_scalar('episode_reward',reward,i_episode)

        # print(next_state)
        # print(state)
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()
        
        # Update the target network, copying all weights and biases in DQN
        if i_episode % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

        if (i_episode+1)%VAL_STEP == 0 or (i_episode == n_steps-1):
            val(i_episode, gym_env_val.reset())
        
    writer.add_hparams(
        {'BATCH_SIZE' : BATCH_SIZE,
        'GAMMA' : GAMMA,
        'EPS_START' : EPS_START,
        'EPS_END' : EPS_END,
        'EPS_DECAY' : EPS_DECAY,
        'TARGET_UPDATE' :TARGET_UPDATE},
        {
            'Overall Reward':ovr_reward,
            'Average Reward': ovr_reward/n_steps
        }
    )

    print(f'Overall Train reward = {ovr_reward:.2f}. ' \
        f'Average Reward = {ovr_reward/n_steps:.4f}')
    gym_env.close()

/home/mohit/anaconda3/envs/ITU/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Train:   0%|          | 0/68700 [00:00<?, ?it/s]

KeyboardInterrupt: 